This notebook computes F-measure and draws Desicion boundary via knn for a clustering result given in a json file

In [ ]:
# minimal imports
import torch, json
from ricci_regularization import RiemannianKmeansTools

In [ ]:
k_means_setup_number = 2
pretrained_AE_setting_name = 'MNIST_Setting_1_exp2'
knn_neighbours_count = 7
point_size = 100
background_opacity = 0.4
colormap = 'jet'

In [ ]:
Path_clustering_setup = f"../experiments/{pretrained_AE_setting_name}/K_means_setup_{k_means_setup_number}"
with open(Path_clustering_setup + f"/params.json", "r") as f_Riemannian:
    Riemannian_k_means_params = json.load(f_Riemannian)
with open(Path_clustering_setup + f"/Euclidean_k_means_params.json", "r") as f_Euclidean:
    Euclidean_k_means_params = json.load(f_Euclidean)
contour_levels_count = Euclidean_k_means_params["K"]+1 # should be k+1

In [ ]:
ground_truth_labels = torch.tensor(Riemannian_k_means_params['ground_truth_labels'])
# check that ground truth labels are saved correctly (are the same in both methods)
assert torch.equal(ground_truth_labels,
                    torch.tensor(Euclidean_k_means_params['ground_truth_labels']))

labels_assigned_by_Euclidean_k_means = torch.tensor(Euclidean_k_means_params['Euclidean_k_means_labels'])
labels_assigned_by_Riemannian_k_means = torch.tensor(Riemannian_k_means_params['Riemannian_k_means_labels'])
encoded_points = torch.tensor(Riemannian_k_means_params['encoded_points'])
assert torch.equal(encoded_points,
                   torch.tensor(Euclidean_k_means_params['encoded_points']))
selected_labels = Riemannian_k_means_params["selected_labels"]

# F_measure

In [ ]:
f_measure = RiemannianKmeansTools.compute_f_measure(labels_assigned_by_Riemannian_k_means, 
            ground_truth_labels)
print("F-measure Riemannian:", f_measure)
open(f"{Path_clustering_setup}/f_measure_Riemannian.txt", "w").write(str(f_measure))
print(f"F-measure Riemannian saved in {Path_clustering_setup}/f_measure_Riemannian.txt")

f_measure = RiemannianKmeansTools.compute_f_measure(labels_assigned_by_Euclidean_k_means, 
            ground_truth_labels)
print("F-measure Euclidean:", f_measure)
open(f"{Path_clustering_setup}/f_measure_Euclidean.txt", "w").write(str(f_measure))
print(f"F-measure Euclidean saved in {Path_clustering_setup}/f_measure_Euclidean.txt")

# Voronoi cells

In [ ]:
# labels by ground truth labels
RiemannianKmeansTools.plot_knn_decision_boundary(encoded_points,
        labels_for_coloring=ground_truth_labels,contour_levels_count=contour_levels_count,
        neighbours_number=knn_neighbours_count, selected_labels=selected_labels,
        saving_folder=Path_clustering_setup, cmap_background=colormap,cmap_points=colormap,
        background_opacity = background_opacity, points_size = point_size,
        plot_title= f"Points in latent space colored by ground truth labels, \nVoronoi's cells colored by {knn_neighbours_count}-NN.",
        file_saving_name=f"Decision_boundary_ground_truth_labels_{pretrained_AE_setting_name}")

In [ ]:
# labels by the Riemannian clustering 
RiemannianKmeansTools.plot_knn_decision_boundary(encoded_points,
        labels_for_coloring=labels_assigned_by_Riemannian_k_means, contour_levels_count=contour_levels_count,
        neighbours_number=knn_neighbours_count, selected_labels=selected_labels,
        points_size=point_size,background_opacity=background_opacity,
        saving_folder=Path_clustering_setup, cmap_background=colormap,cmap_points=colormap,
        plot_title= f"Points in latent space colored by labels assigned by\n Riemannian k-means, Voronoi's cells colored by {knn_neighbours_count}-NN.",
        file_saving_name=f"Decision_boundary_Riemannian_k-means_labels_{pretrained_AE_setting_name}")

In [ ]:
# labels by the Euclidean clustering 
RiemannianKmeansTools.plot_knn_decision_boundary(encoded_points,
        labels_for_coloring=labels_assigned_by_Euclidean_k_means, contour_levels_count=contour_levels_count,
        neighbours_number=knn_neighbours_count, selected_labels=selected_labels,
        points_size=point_size,background_opacity=background_opacity,
        saving_folder=Path_clustering_setup, cmap_background=colormap,cmap_points=colormap,
        plot_title= f"Points in latent space colored by labels assigned by\n Euclidean k-means, Voronoi's cells colored by {knn_neighbours_count}-NN.",
        file_saving_name=f"Decision_boundary_Euclidean_k-means_labels_{pretrained_AE_setting_name}")